## In this notebook, we are going to cover some of the basics of tensors with tensorflow.

1. Introduction to Tensors
2. Getting information from tensors
3. Manipulating tensors
4. Tensors and NumPy
5. Using @tf.function (a way to speed up your regular Python functions)
6. Using GPUs with TensorFlow (or TPUs)
7. Exercises to try for yourself!

## Introduction to Tensors

In [1]:
#Import tensorflow
import tensorflow as tf

print(tf.version.VERSION)

2024-05-07 06:38:05.128746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.10.0


In [2]:
# Create tensor with tf.constant() 
scalar = tf.constant(7)
scalar

2024-05-06 20:17:25.742491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimensions of a tensor
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# Check the number of dimensions of a tensor
vector.ndim

1

In [6]:
# Create a matrix
matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
# Check the number of dimensions of a tensor
matrix.ndim

2

In [8]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type with dtype parameter
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# Check the number of dimensions of a tensor
another_matrix.ndim

2

In [10]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                      [4, 5, 6]],
                     [[7, 8, 9],
                     [10, 11, 12]],
                     [[13, 14, 15],
                     [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
# Check the number of dimensions of a tensor
tensor.ndim

3

# What we've created so far
* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

In [12]:
tf.Variable

tensorflow.python.ops.variables.Variable

### Creating tensors with tf.Variable

In [13]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [14]:
# Change one of the elements in our changeable tensor
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [15]:
# Let's try to change our unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

## Creating random tensors

Random tensors are tensors of arbitrary size which contains random numbers

In [16]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))

random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_2, random_1 == random_2


(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [17]:
# Shuffle a tensor (valuable for when you want to shuffle your data)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
not_shuffled.ndim

2

In [18]:
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

In [19]:
# If we want our shuffled tensors to be in the same order, 
# we've got to use the global level random seed as well as the operation level random seed:

# Set the global random seed
tf.random.set_seed(42)

# Set the operation random seed
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [20]:
# Homework
random3 = tf.random.Generator.from_seed(42)
random3 = random3.normal(shape=(3, 2))
random3

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [21]:
# Now shuffle random3 (always constant since using global and operation level seed)
tf.random.set_seed(42)
tf.random.shuffle(random3, seed=42)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

## Other ways to make tensors

In [22]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [23]:
# Create a tensor of all zeros
tf.zeros(shape=(3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing).

In [24]:
# You can turn NumPy arrays into tensors
import numpy as np

numpy_A = np.arange(1, 25, dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [25]:
A = tf.constant(numpy_A, shape=[2, 3, 4])
A

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

### Getting information from tensors

When dealing with tensors, you probably want to be aware of the following attributes
* Shape
* Rank
* Axis or dimension
* Size

In [26]:
# Create a rank 4 tensor (4 dimensions) 

rank_4_tensor = tf.zeros([2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [27]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [28]:
# Get various attributes of our tensor
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [29]:
# Shape of rank_4_tensor multiplied must equal number of elements in rank_4_tensor
2 * 3 * 4 * 5

120

In [30]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: 120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [31]:
some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [32]:
# Get the first 2 items of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [33]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [34]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7], [3, 4]])
rank_2_tensor, rank_2_tensor.shape, rank_2_tensor.ndim

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 3,  4]], dtype=int32)>,
 TensorShape([2, 2]),
 2)

In [35]:
# Get last item of each row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [36]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [37]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand the final axis


<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

### Manipulating tensors (tensor operations)

** Basic operations

+, -, *, /

In [38]:
# You can add values using the addition operator

tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [39]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [40]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [41]:
# Subtraction if you want
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [42]:
# Division
tensor / 10

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.7],
       [0.3, 0.4]])>

In [43]:
# We can use the tensorflow built-in function too
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

## Matrix multiplication in TensorFlow

In machine learning, matrix multiplication is one of the most common tensor operations.

1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [44]:
# Matrix multiplication in tensorflow
tf.matmul(tensor, tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [45]:
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [46]:
tensor1 = tf.constant([[1, 2, 5], [7, 2, 1], [3, 3, 3]])
tensor2 = tf.constant([[3, 5], [6, 7], [1, 8]])
tf.matmul(tensor1, tensor2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [47]:
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [48]:
# Create a tensor (3, 2) tensor
X = tf.constant([[1, 2], [3, 4], [5, 6]])

# Create another tensor (3, 2) tensor
Y = tf.constant([[7, 8], [9, 10], [11, 12]])

In [49]:
# Try to multiply tensors of same shape
# X * Y

### Resource and examples of matrix multiplications
https://www.mathsisfun.com/algebra/matrix-multiplying.html

In [50]:
print(X.shape, Y.shape)

# Change the shape of Y
Y = tf.reshape(Y, shape=(2, 3))

tf.matmul(X, Y)

(3, 2) (3, 2)


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [51]:
# Try to multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [52]:
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [53]:
# Can do the same with transpose
print(X, Y)
tf.transpose(X), tf.transpose(Y)    

tf.Tensor(
[[1 2]
 [3 4]
 [5 6]], shape=(3, 2), dtype=int32) tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32)


(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7, 10],
        [ 8, 11],
        [ 9, 12]], dtype=int32)>)

**The Dot Product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`

In [54]:
print(X, Y)

tf.Tensor(
[[1 2]
 [3 4]
 [5 6]], shape=(3, 2), dtype=int32) tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32)


In [55]:
X = tf.transpose(X)

In [56]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [60]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 76, 103],
       [100, 136]], dtype=int32)>

In [62]:
X = tf.transpose(X)

In [63]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [64]:
# Check the value of Y, reshape Y, and transpose Y
print("Normal Y:", Y)
print("\n")
print("Y reshaped to (2, 3):", tf.reshape(Y, (2, 3)))
print("\n")
print("Y transposed:", tf.transpose(Y))


Normal Y: tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32)


Y reshaped to (2, 3): tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32)


Y transposed: tf.Tensor(
[[ 7 10]
 [ 8 11]
 [ 9 12]], shape=(3, 2), dtype=int32)


Generally, when performing matrix multiplication on two tensors and one of the axes 
doesn't line up, you will need to transpose (rather than reshape) one of the tensors 
to satisfy the matrix multiplication rules.

## Changing the datatype of tensor

In [65]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [66]:
C = tf.constant([1, 7])
C.dtype

tf.int32

In [67]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [68]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 7.], dtype=float32)>

In [69]:
E_float16 = tf.cast(E, dtype=tf.float16)

## Aggregating tensors

Aggregating tensors = condensing multiple values down to a smaller amount of values.  

In [70]:
# Getting the absolute values
D = tf.constant([-7, -10])
print(D)
tf.abs(D)

tf.Tensor([ -7 -10], shape=(2,), dtype=int32)


<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregation:

* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [77]:
E = tf.constant(np.random.randint(0, 100, size=50), dtype=tf.float32)
tf.size(E), E.shape, E.ndim




(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [78]:
# Find the minimum, maximum, mean, and sum of our tensor
tf.reduce_min(D), tf.reduce_max([[1, 7], [3, 4]]), tf.reduce_mean([1, 2, 3]), tf.reduce_sum([1, 2, 3])

(<tf.Tensor: shape=(), dtype=int32, numpy=-10>,
 <tf.Tensor: shape=(), dtype=int32, numpy=7>,
 <tf.Tensor: shape=(), dtype=int32, numpy=2>,
 <tf.Tensor: shape=(), dtype=int32, numpy=6>)

In [79]:
# Find the variance and standard deviation of our tensor
tf.math.reduce_variance(E), tf.math.reduce_std(E)

(<tf.Tensor: shape=(), dtype=float32, numpy=642.2544>,
 <tf.Tensor: shape=(), dtype=float32, numpy=25.34274>)

In [80]:
import tensorflow_probability as tfp

In [84]:
tfp.stats.variance(tf.cast(E, dtype=tf.float32)), tfp.stats.stddev(tf.cast(E, dtype=tf.float32))

(<tf.Tensor: shape=(), dtype=float32, numpy=642.2544>,
 <tf.Tensor: shape=(), dtype=float32, numpy=25.34274>)

In [94]:
# Create a new tensor for finding positional maximum and minimum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50], seed=42)
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.4163028 , 0.26858163, 0.47968316, 0.36457133, 0.95471144,
       0.9418646 , 0.61483395, 0.35842144, 0.5936024 , 0.21551096,
       0.07745171, 0.57921314, 0.29180396, 0.26718032, 0.37012458,
       0.7161033 , 0.45877767, 0.11764562, 0.21073711, 0.5441973 ,
       0.9898069 , 0.38395858, 0.04683566, 0.8718462 , 0.25881708,
       0.873135  , 0.64698434, 0.41981232, 0.24148273, 0.09550059,
       0.9820819 , 0.1570208 , 0.2997682 , 0.36795306, 0.9453716 ,
       0.11056781, 0.52287626, 0.8305441 , 0.0020721 , 0.9594034 ,
       0.85630023, 0.3944497 , 0.22028875, 0.67066073, 0.01875746,
       0.48057055, 0.5953454 , 0.6847329 , 0.18988943, 0.12489867],
      dtype=float32)>

In [96]:
# Find the positional maximum and minimum
tf.argmax(F), tf.argmin(F)

(<tf.Tensor: shape=(), dtype=int64, numpy=20>,
 <tf.Tensor: shape=(), dtype=int64, numpy=38>)

In [97]:
# Index on our largest value position
F[tf.argmax(F)], F[tf.argmin(F)]

(<tf.Tensor: shape=(), dtype=float32, numpy=0.9898069>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0020720959>)

In [98]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F), F[tf.argmin(F)] == tf.reduce_min(F)

(<tf.Tensor: shape=(), dtype=bool, numpy=True>,
 <tf.Tensor: shape=(), dtype=bool, numpy=True>)

In [100]:
# Create a tensor
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50], seed=42), shape=(1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 50), dtype=float32, numpy=
array([[[[0.4163028 , 0.26858163, 0.47968316, 0.36457133, 0.95471144,
          0.9418646 , 0.61483395, 0.35842144, 0.5936024 , 0.21551096,
          0.07745171, 0.57921314, 0.29180396, 0.26718032, 0.37012458,
          0.7161033 , 0.45877767, 0.11764562, 0.21073711, 0.5441973 ,
          0.9898069 , 0.38395858, 0.04683566, 0.8718462 , 0.25881708,
          0.873135  , 0.64698434, 0.41981232, 0.24148273, 0.09550059,
          0.9820819 , 0.1570208 , 0.2997682 , 0.36795306, 0.9453716 ,
          0.11056781, 0.52287626, 0.8305441 , 0.0020721 , 0.9594034 ,
          0.85630023, 0.3944497 , 0.22028875, 0.67066073, 0.01875746,
          0.48057055, 0.5953454 , 0.6847329 , 0.18988943, 0.12489867]]]],
      dtype=float32)>

In [101]:
G.shape, G.ndim

(TensorShape([1, 1, 1, 50]), 4)

In [103]:
G_squeezed = tf.squeeze(G)
G_squeezed

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.4163028 , 0.26858163, 0.47968316, 0.36457133, 0.95471144,
       0.9418646 , 0.61483395, 0.35842144, 0.5936024 , 0.21551096,
       0.07745171, 0.57921314, 0.29180396, 0.26718032, 0.37012458,
       0.7161033 , 0.45877767, 0.11764562, 0.21073711, 0.5441973 ,
       0.9898069 , 0.38395858, 0.04683566, 0.8718462 , 0.25881708,
       0.873135  , 0.64698434, 0.41981232, 0.24148273, 0.09550059,
       0.9820819 , 0.1570208 , 0.2997682 , 0.36795306, 0.9453716 ,
       0.11056781, 0.52287626, 0.8305441 , 0.0020721 , 0.9594034 ,
       0.85630023, 0.3944497 , 0.22028875, 0.67066073, 0.01875746,
       0.48057055, 0.5953454 , 0.6847329 , 0.18988943, 0.12489867],
      dtype=float32)>

### One-hot encoding tensors

In [106]:
# Create list of indices
some_list = [0, 1, 2, 3]  # could be red, green, blue, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [107]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="yo I love deep learning", off_value="I also like to dance")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'yo I love deep learning', b'I also like to dance',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'yo I love deep learning',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'yo I love deep learning', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'I also like to dance', b'yo I love deep learning']],
      dtype=object)>

## Squaring, log, square root

In [109]:
# Creating a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [110]:
# Square the tensor
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [111]:
# Find square root
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [112]:
# Find the log
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

## Tensor and Numpy

TensorFlow interacts beautifully with Numpy Arrays

In [114]:
J = tf.constant(np.array([3. ,7. ,10. ]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [116]:
# Convert tensor J to a numpy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [118]:
# Convert tensor J to a list
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [119]:
J = tf.constant([3.])
J.numpy()

array([3.], dtype=float32)

In [120]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_j = tf.constant([3., 7., 10.])

# Check the datatype of each
numpy_J.dtype, tensor_j.dtype

(tf.float64, tf.float32)

### Finding access to GPUs

In [121]:
# Check what device we're using
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Exercises

In [6]:
# Create scalar, vector, matrix, and tensor
scalar = tf.constant(5)
vector = tf.constant([1,2,3])
matrix = tf.constant([[1,2,3], [1,2,3]])
tensor = tf.constant([[[1,2,3], [1,2,3]], [[4,5,6], [4,5,6]]])

scalar, vector, matrix, tensor


(<tf.Tensor: shape=(), dtype=int32, numpy=5>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [1, 2, 3]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
 array([[[1, 2, 3],
         [1, 2, 3]],
 
        [[4, 5, 6],
         [4, 5, 6]]], dtype=int32)>)

In [9]:
# Find shape, rank, and dimension
print(scalar.shape, tf.rank(scalar), scalar.ndim)
print(vector.shape, tf.rank(vector), vector.ndim)
print(matrix.shape, tf.rank(matrix), matrix.ndim)
print(tensor.shape, tf.rank(tensor), tensor.ndim)

() tf.Tensor(0, shape=(), dtype=int32) 0
(3,) tf.Tensor(1, shape=(), dtype=int32) 1
(2, 3) tf.Tensor(2, shape=(), dtype=int32) 2
(2, 2, 3) tf.Tensor(3, shape=(), dtype=int32) 3


In [13]:
# Create two random tensors with shape [5, 300] between 0 and 1
tf.random.set_seed(42)
random_1 = tf.random.normal([5, 300], 0, 1, seed=42)
random_2 = tf.random.normal([5, 300], 0, 1, seed=43)

random_1, random_2

(<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[ 1.3148774 , -0.15421568,  0.9113878 , ...,  1.2660782 ,
          0.20770869,  0.04344311],
        [-0.02998826,  0.20488335, -0.75734955, ...,  0.8893339 ,
         -0.46456474, -0.25324404],
        [ 0.35314846,  0.7992969 , -1.0041809 , ...,  0.04357555,
         -1.3240569 ,  0.36490098],
        [-1.2563272 , -1.639324  ,  0.02165308, ..., -0.7794096 ,
         -0.00282614, -0.4324933 ],
        [-0.7633545 ,  0.239836  , -0.15629388, ..., -0.70517975,
          1.4939045 ,  0.58036065]], dtype=float32)>,
 <tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[-1.425722  , -1.0143788 ,  0.3938772 , ..., -0.07171483,
         -0.5966395 ,  0.17834884],
        [ 1.8681601 , -0.51730883, -0.71657765, ...,  0.09107281,
          0.65695065,  0.95911634],
        [-2.2293987 ,  0.4139686 , -1.3875371 , ..., -0.74282664,
          1.1414378 , -1.7282356 ],
        [ 0.75614357, -0.5512401 , -1.4140644 , ...,  0.13298689

In [15]:
# Matrix Multiplication
random_1 * random_2

<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
array([[-1.87464964e+00,  1.56433105e-01,  3.58974874e-01, ...,
        -9.07965899e-02, -1.23927213e-01,  7.74802873e-03],
       [-5.60228787e-02, -1.05987966e-01,  5.42699754e-01, ...,
         8.09941366e-02, -3.05196106e-01, -2.42890492e-01],
       [-7.87308753e-01,  3.30883831e-01,  1.39333832e+00, ...,
        -3.23690809e-02, -1.51132846e+00, -6.30634844e-01],
       [-9.49963689e-01,  9.03661072e-01, -3.06188520e-02, ...,
        -1.03651255e-01, -2.06790701e-03, -2.10483924e-01],
       [-8.89155269e-01, -4.78292048e-01, -1.57193333e-01, ...,
        -8.45736742e-01,  3.33519554e+00, -7.33018875e-01]], dtype=float32)>

In [17]:
# Dot Product 
tf.matmul(random_1, tf.transpose(random_2))


<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[-10.419104  ,  28.059113  , -34.840443  ,  23.04887   ,
         33.695545  ],
       [ -5.3592253 ,  14.165163  , -18.561731  ,   7.0949473 ,
        -32.785343  ],
       [-22.692156  ,   0.529335  ,   0.14620018,  23.29767   ,
          4.0824437 ],
       [ 14.72619   ,  13.604471  ,  10.400473  ,  16.901148  ,
         -5.0640965 ],
       [ 12.918938  , -18.071451  , -12.026243  ,  12.689505  ,
         -9.479393  ]], dtype=float32)>

In [18]:
# Create another random tensor
random_3 = tf.random.normal([224, 224, 3], 0, 1)
random_3

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 0.3274685 , -0.8426258 ,  0.3194337 ],
        [-1.4075519 , -2.3880599 , -1.0392479 ],
        [-0.5573232 ,  0.539707  ,  1.6994323 ],
        ...,
        [-2.1195276 ,  1.8320248 ,  1.1457133 ],
        [ 0.54631245,  1.8743154 ,  0.11950399],
        [-0.29805267, -0.29960835,  1.5552508 ]],

       [[-0.3161619 , -0.72182304, -0.98977196],
        [ 1.1060055 , -0.21110982, -0.47060865],
        [ 0.07136566, -0.48230812, -1.2975956 ],
        ...,
        [ 0.10062912,  0.38567653, -0.19041471],
        [-1.127596  ,  0.64350545,  1.1970826 ],
        [-0.3737931 ,  0.2361026 , -0.13297646]],

       [[ 1.3060379 ,  0.26648018, -0.07864442],
        [-1.114526  , -0.595572  ,  0.71523064],
        [ 1.2495805 ,  0.38960335, -0.5431543 ],
        ...,
        [ 1.616052  ,  0.88652635,  1.0241655 ],
        [ 0.26582795,  0.82782304, -0.27606866],
        [ 1.8379737 , -0.24267258,  0.62618935]],

       ...,

     

In [19]:
# Find min and max
tf.reduce_min(random_3), tf.reduce_max(random_3)

(<tf.Tensor: shape=(), dtype=float32, numpy=-5.4462285>,
 <tf.Tensor: shape=(), dtype=float32, numpy=4.486006>)

In [20]:
# Yet another random
random_4 = tf.random.normal([1, 224, 224, 3], 0, 1)
random_4

<tf.Tensor: shape=(1, 224, 224, 3), dtype=float32, numpy=
array([[[[ 8.42245817e-02, -8.60903740e-01,  3.78123045e-01],
         [-5.19627379e-03, -4.94531959e-01,  6.17819190e-01],
         [-3.30820471e-01, -1.38408062e-03, -4.23734099e-01],
         ...,
         [ 7.27746725e-01, -1.18733458e-02,  3.83172005e-01],
         [-6.53265953e-01,  9.58728135e-01, -1.26495326e+00],
         [-1.15851891e+00, -1.09168744e+00, -1.24350893e+00]],

        [[ 3.94207805e-01,  9.73992944e-01,  1.27714586e+00],
         [-3.69007319e-01, -1.95457017e+00,  1.26128745e+00],
         [ 4.01002795e-01, -8.19520950e-02, -1.12813346e-01],
         ...,
         [ 3.55617613e-01, -2.48408034e-01, -2.91345358e-01],
         [ 3.71442884e-02, -4.43115622e-01,  1.49573183e+00],
         [-5.19530952e-01, -2.23524499e+00,  6.92982316e-01]],

        [[ 1.13621640e+00, -5.89942157e-01, -2.21491838e+00],
         [-1.41675687e+00,  9.28691506e-01, -8.90731156e-01],
         [-7.77718604e-01,  3.26967597e-01

In [21]:
tf.squeeze(random_4)

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 8.42245817e-02, -8.60903740e-01,  3.78123045e-01],
        [-5.19627379e-03, -4.94531959e-01,  6.17819190e-01],
        [-3.30820471e-01, -1.38408062e-03, -4.23734099e-01],
        ...,
        [ 7.27746725e-01, -1.18733458e-02,  3.83172005e-01],
        [-6.53265953e-01,  9.58728135e-01, -1.26495326e+00],
        [-1.15851891e+00, -1.09168744e+00, -1.24350893e+00]],

       [[ 3.94207805e-01,  9.73992944e-01,  1.27714586e+00],
        [-3.69007319e-01, -1.95457017e+00,  1.26128745e+00],
        [ 4.01002795e-01, -8.19520950e-02, -1.12813346e-01],
        ...,
        [ 3.55617613e-01, -2.48408034e-01, -2.91345358e-01],
        [ 3.71442884e-02, -4.43115622e-01,  1.49573183e+00],
        [-5.19530952e-01, -2.23524499e+00,  6.92982316e-01]],

       [[ 1.13621640e+00, -5.89942157e-01, -2.21491838e+00],
        [-1.41675687e+00,  9.28691506e-01, -8.90731156e-01],
        [-7.77718604e-01,  3.26967597e-01,  1.99382842e-01],


In [61]:
tf.random.set_seed(42)
random_5 = tf.random.uniform([10], 0, 10, seed=42, dtype=tf.int32)
random_5

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([3, 8, 0, 4, 8, 1, 3, 3, 2, 5], dtype=int32)>

In [62]:
# Find positional max and min
tf.argmax(random_5), tf.argmin(random_5)

(<tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>)

In [63]:
indices = random_5
depth = 10

tf.one_hot(indices, depth)

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)>